### Analyze LangSmith Datasets with Lilac

Lilac is an open-source product that helps you analyze, structure, and clean unstructured data with AI. 

Basic overview:
- Create dataset from runs
- Visualize the data
- Query the dataset
- Download the dataset

In [ ]:
# %pip install -U "lilac[pii]" langdetect sentence-transformers langsmith --quiet

## Step 1: Create dataset of runs

In [10]:
# We'll start by fetching the root traces from a project
from langsmith import Client
from datetime import datetime, timedelta

client = Client()

project_name = "chat-langchain"
start_time = datetime.now() - timedelta(days=7)


runs = list(client.list_runs(
    project_name=project_name,
    start_time=start_time,
    execution_order=1,
    run_type="chain",
))
len(runs)

1621

Now let's create the dataset. We'll flatten some of the fields out to make it easiert to work with in Lilac.

In [11]:
from concurrent.futures import ThreadPoolExecutor
import json

dataset_name = f"{project_name}_EDA_dataset"
# client.delete_dataset(dataset_name=dataset_name)
dataset = client.create_dataset(
    dataset_name=dataset_name,
)

with ThreadPoolExecutor(max_workers=30) as executor:
    executor.map(
        lambda run: client.create_example(
            inputs={
                # Lilac may have some issues on deeply nested structures
                **{k: json.dumps(v) for k, v in run.inputs.items()},
                "run_name": run.name,
                "latency": (run.end_time - run.start_time).total_seconds(),
            },
            outputs={
                **{k: json.dumps(v) for k, v in (run.outputs or {}).items()},
                "error": str(run.error)
            },
            dataset_id=dataset.id,
        ), 
        runs
    )

### From LangSmith Dataset

Let's create a Lilac dataset from a LangSmith dataset.

In [3]:
from IPython.display import display
import lilac as ll

In [12]:
data_source = ll.sources.langsmith.LangSmithSource(
    dataset_name=dataset_name,
)

config = ll.DatasetConfig(
  namespace='local',
  name=dataset_name,
  source=data_source,
)

dataset = ll.create_dataset(config)

Reading from source langsmith...: 100%|██████████████████████████████████| 1616/1616 [00:00<00:00, 217354.90it/s]

Dataset "chat-langchain_EDA_dataset" written to data/datasets/local/chat-langchain_EDA_dataset


## Visualize the data

Now that we have imported a few datasets, let's visualize them to see what they look like.


In [19]:
ll.start_server(project_path='data')
await ll.stop_server()

INFO:     Started server process [63729]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)


Scheduling task "90bf7d86052b4978916f0181302a7fec": "[local/chat-langchain_EDA_dataset] Compute signal "openai" on "input"".
Route error: http://127.0.0.1:5432/api/v1/datasets/local/chat-langchain_EDA_dataset/compute_signal
Tried sending message after closing.  Status: closed
Message: {'op': 'update-graph', 'graph_header': {'serializer': 'pickle', 'writeable': ()}, 'graph_frames': [b'\x80\x05\x95\x0b\x08\x00\x00\x00\x00\x00\x00\x8c\x1edistributed.protocol.serialize\x94\x8c\x08ToPickle\x94\x93\x94)\x81\x94}\x94\x8c\x04data\x94\x8c\x13dask.highlevelgraph\x94\x8c\x0eHighLevelGraph\x94\x93\x94)\x81\x94}\x94(\x8c\x0cdependencies\x94}\x94\x8a\x05@\x1d\x12\xd1\x02\x8f\x94s\x8c\x10key_dependencies\x94}\x94\x8c\x06layers\x94}\x94\x8a\x05@\x1d\x12\xd1\x02h\x06\x8c\x11MaterializedLayer\x94\x93\x94)\x81\x94}\x94(\x8c\x0bannotations\x94N\x8c\x16collection_annotations\x94N\x8c\x07mapping\x94}\x94\x8c 90bf7d86052b4978916f0181302a7fec\x94(\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94\x8c\x0blilac.tas

In [15]:
# Navigate to the dataset. It should be visible at
"http://127.0.0.1:5432/datasets"

'http://127.0.0.1:5432/datasets'

## View Dataset Schema

The Lil

In [ ]:
# Show the dataset schema
dataset.manifest()

## Enriching an unstructured field with metadata

Lilac exposes a number of built-in methods to to add structured metadata to your dataset.
Called "signals", these methods compute a function on each row and add the results as new fields
to the field on which they were applied.

In this example, we will run a "signal" over the `question` field.

In [ ]:
dataset.compute_signal(ll.LangDetectionSignal(), 'question')

In [ ]:
# Apply min-hash LSH (https://en.wikipedia.org/wiki/MinHash) to detect approximate n-gram duplicates
dataset.compute_signal(ll.NearDuplicateSignal(), 'question')

# Query the Dataset

Now that we've enriched the dataset, we can query it to explore it.

In [ ]:
r = dataset.select_rows(['question', 'answer'], limit=5)
r.df()

## Searching


### Compute embedding to enable advanced search

Let's compute the `SBERT` embedding on device for the `overview` field.


In [ ]:
dataset.compute_embedding('sbert', 'question')

### Keyword search


In [ ]:
query = ll.KeywordQuery(search='runnable')
r = dataset.select_rows(['question'], searches=[ll.Search(path='question', query=query)], limit=5)
display(r.df())

In [ ]:
ll.

### Semantic search


In [ ]:
query = ll.SemanticQuery(search='runnable', embedding='sbert')
r = dataset.select_rows(['overview'], searches=[ll.Search(path='overview', query=query)], limit=5)
display(r.df())

### Conceptual search


In [ ]:
query = ll.ConceptQuery(concept_namespace='lilac', concept_name='profanity', embedding='sbert')
r = dataset.select_rows(['overview'], searches=[ll.Search(path='overview', query=query)], limit=5)
display(r.df())

## Downloading the enriched dataset


In [ ]:
dataset.to_csv('the_movies_dataset.csv')

In [ ]:
dataset.to_pandas()[:5]

## Using concepts


### Use the positive-sentiment concept


In [ ]:
signal = ll.signals.ConceptSignal(
  namespace='lilac', concept_name='positive-sentiment', embedding='gte-small')

result = list(signal.compute(['This product is amazing, thank you!']))

print(result)


### Create a positive product reviews concept


In [ ]:
db = ll.DiskConceptDB()

concepts = db.list()
# Don't create the concept twice.
if not list(
    filter(lambda c: c.namespace == 'local' and c.name == 'positive-product-reviews', concepts)):
  db.create('local', 'positive-product-reviews')

#### Add a few training examples


In [ ]:
train_data = [
  ll.ExampleIn(label=False, text='The quick brown fox jumps over the lazy dog.'),
  ll.ExampleIn(label=False, text='This is a random sentence.'),
  ll.ExampleIn(label=True, text='This product is amazing!'),
  ll.ExampleIn(label=True, text='Thank you for your awesome work on this UI.')
]
db.edit('local', 'positive-product-reviews', ll.ConceptUpdate(insert=train_data))

#### Show the examples in the concept


In [ ]:
concept = db.get('local', 'positive-product-reviews')

if concept:
  print(concept.data)

#### Remove examples


In [ ]:
db.edit('local', 'positive-product-reviews',
        ll.ConceptUpdate(remove=['d86e4cb53c70443b8d8782a6847f4752']))

##### Use the new concept


In [ ]:
signal = ll.signals.ConceptSignal(
  namespace='local', concept_name='positive-product-reviews', embedding='gte-small')

result = list(signal.compute(['This product is amazing, thank you!']))

print(result)

#### Concept metrics

To compute metrics for a concept, we first have to instantiate a concept model.


In [ ]:
model_db = ll.DiskConceptModelDB(ll.DiskConceptDB())

model = model_db.get('local', 'positive-product-reviews', embedding_name='gte-small')

if model:
  print(model.get_metrics())


#### Remove the concept


In [ ]:
db.remove('local', 'positive-product-reviews')